### 📄 **Profile: `scalar-raster`**

The `scalar-raster` profile defines a CF-compliant structure for datasets composed of one or more 2D raster variables in Zarr format, designed for geospatial scalar fields such as elevation, temperature, or land cover.

The dataset **must contain**:
- A global attribute **`Conventions = "CF-1.8"`** (mandatory) and **`profile = "scalar-raster"`** (recommended) to identify the profile.
- One or more data variables representing 2D scalar fields, each defined on dimensions `(y, x)`.
- Coordinate variables named **`x`** and **`y`**, each with required CF attributes:
  - `x`: `standard_name = "projection_x_coordinate"`, `units = "metre"`
  - `y`: `standard_name = "projection_y_coordinate"`, `units = "metre"`

Each data variable **must** include:
- A valid CF **`standard_name`** describing the physical quantity (e.g. `surface_temperature`)
- A corresponding **`units`** attribute
- A **`grid_mapping`** attribute referencing the spatial reference variable

The **grid mapping variable** is required and defines the projection using CF-compliant attributes. The variable name **`spatial_ref`** is recommended for consistency.

Additional information:
- The group may contain nested groups, allowing namespacing or grouping of related variables.
- Coordinates can be provided either as explicit arrays (`x`, `y`) or via an affine transform, if supported (subject to profile implementation discussion).

Chunking layout is implementation-dependent and not prescribed by the profile. The focus is on naming, structure, and semantic metadata required for CF compliance.

In [9]:
import xarray as xr
scalar = xr.open_zarr("./scalar-raster.zarr", consolidated=False)
scalar

<xarray.Dataset> Size: 133kB
Dimensions:      (y: 128, x: 128)
Coordinates:
  * x            (x) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
  * y            (y) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
Data variables:
    scalar_data  (y, x) float64 131kB dask.array<chunksize=(128, 128), meta=np.ndarray>
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    profile:      scalar-raster

### 📄 **Profile: `rgb-raster**

The `rgb-raster` profile defines a CF-compliant structure for multi-band raster datasets in Zarr format, tailored for Earth Observation imagery.

The dataset **must contain**:
- A global attribute **`Conventions = "CF-1.8"`**  and **`profile = "rgb-raster"`** are recommended to identify the profile.
- A data variable named **`rgb_data`** representing the raster array, with dimensions `(band, y, x)`.
- Coordinate variables named **`x`**, **`y`**, and **`band`**, each with required CF attributes:
  - `x`: `standard_name = "projection_x_coordinate"`, `units = "metre"`
  - `y`: `standard_name = "projection_y_coordinate"`, `units = "metre"`
  - `band`: `standard_name = "sensor_band_identifier"`, no units required

The data variable `rgb_data` **must** include:
- A valid CF **`standard_name`** representing the physical quantity (e.g. reflectance, radiance)
- A corresponding **`units`** attribute
- A **`grid_mapping`** attribute referencing the spatial reference variable

The **grid mapping variable** is required and defines the projection using CF-compliant attributes. The variable name **`spatial_ref`** is recommended for consistency.

Additional information:
- The group may contain nested groups, allowing namespacing or grouping of related variables.
- Coordinates can be provided either as explicit arrays (x, y) or via an affine transform, if supported (subject to profile implementation discussion).


Chunking layout is implementation-dependent and not prescribed by the profile. The focus is on naming, structure, and semantic metadata required for CF compliance.

In [10]:
import xarray as xr
rgb = xr.open_zarr("./rgb-raster.zarr", consolidated=False)
rgb


<xarray.Dataset> Size: 133kB
Dimensions:      (band: 1, y: 128, x: 128)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 1kB 3.193e+05 3.193e+05 ... 3.209e+05 3.209e+05
  * y            (y) float64 1kB 5.639e+06 5.639e+06 ... 5.638e+06 5.638e+06
Data variables:
    rgb_data     (band, y, x) float64 131kB dask.array<chunksize=(1, 64, 64), meta=np.ndarray>
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    profile:      rgb-raster

In [5]:
# Group-level attributes
print("Attributes:")
print(ds.attrs)

# Dimensions
print("\nDimensions:")
print(ds.dims)
print("\nCoordinates (CF names and attributes):")
for coord in ds.coords:
    print(f"- {coord}: {ds.coords[coord].attrs}")
print("\nData Variables (CF names and attributes):")
for var in ds.data_vars:
    print(f"- {var}: {ds[var].attrs}")


Attributes:
{'Conventions': 'CF-1.8', 'profile': 'rgb-raster'}

Dimensions:
FrozenMappingWarningOnValuesAccess({'band': 1, 'y': 128, 'x': 128})

Coordinates (CF names and attributes):
- band: {'long_name': 'spectral band identifier', 'standard_name': 'sensor_band_identifier'}
- x: {'long_name': 'x coordinate of projection', 'standard_name': 'projection_x_coordinate', 'units': 'metre'}
- y: {'long_name': 'y coordinate of projection', 'standard_name': 'projection_y_coordinate', 'units': 'metre'}

Data Variables (CF names and attributes):
- rgb_data: {'AREA_OR_POINT': 'Point', 'TIFFTAG_RESOLUTIONUNIT': '1 (unitless)', 'TIFFTAG_XRESOLUTION': 1, 'TIFFTAG_YRESOLUTION': 1, 'grid_mapping': 'spatial_ref', 'standard_name': 'surface_reflectance', 'units': '1'}
- spatial_ref: {'GeoTransform': '319281.23384248465 12.50000111758709 0.0 5639331.754195504 0.0 -12.50000111758709', 'crs_wkt': 'PROJCS["WGS 84 / UTM zone 30N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORI

### 📄 **Profile : `multispectral-raster`**

The `multispectral-raster` profile defines a CF-compliant structure for multi-band raster datasets in Zarr format, where each band corresponds to a specific spectral wavelength. It is tailored for Earth Observation data containing reflectance or radiance values across multiple spectral regions.

The dataset **must contain**:
- A global attribute **`Conventions = "CF-1.8"`** (mandatory) and **`profile = "multispectral-raster"`** (recommended) to identify the profile.
- A data variable (e.g. **`reflectance`**) representing the raster array, with dimensions `(wavelength, y, x)`.
- Coordinate variables named **`x`**, **`y`**, and **`wavelength`**, each with required CF attributes:
  - `x`: `standard_name = "projection_x_coordinate"`, `units = "metre"`
  - `y`: `standard_name = "projection_y_coordinate"`, `units = "metre"`
  - `wavelength`: `standard_name = "radiation_wavelength"`, `units = "micrometre"`

The data variable (e.g. `reflectance`) **must** include:
- A valid CF **`standard_name`** representing the physical quantity (e.g. `surface_reflectance`)
- A corresponding **`units`** attribute
- A **`grid_mapping`** attribute referencing the spatial reference variable

The **grid mapping variable** is required and defines the projection using CF-compliant attributes. The variable name **`spatial_ref`** is recommended for consistency.

Additional information:
- The group may contain nested groups, allowing namespacing or grouping of related variables.
- Coordinates can be provided either as explicit arrays (`x`, `y`) or via an affine transform, if supported (subject to profile implementation discussion).

Chunking layout is implementation-dependent and not prescribed by the profile. The focus is on naming, structure, and semantic metadata required for CF compliance.

In [12]:
import xarray as xr
multispectral = xr.open_zarr("./multispectral-raster.zarr", consolidated=False)
multispectral

<xarray.Dataset> Size: 1MB
Dimensions:      (wavelength: 8, y: 128, x: 128)
Coordinates:
  * wavelength   (wavelength) float64 64B 0.443 0.49 0.56 ... 1.61 2.2 0.865
  * x            (x) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
  * y            (y) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
Data variables:
    reflectance  (wavelength, y, x) float64 1MB dask.array<chunksize=(4, 64, 128), meta=np.ndarray>
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    profile:      multispectral-raster

### 📄 **Profile: `band-raster`**

The `band-raster` profile defines a CF-compliant structure for multi-band raster datasets in Zarr format, where each band is identified by a discrete sensor band index. It is designed for Earth Observation products that do not directly use physical wavelength as a core dimension but optionally associate bands with their spectral characteristics.

The dataset **must contain**:
- A global attribute **`Conventions = "CF-1.8"`** (mandatory) and **`profile = "band-raster"`** (recommended) to identify the profile.
- A data variable (e.g. **`reflectance`**) representing the raster array, with dimensions `(band, y, x)`.
- Coordinate variables named **`x`**, **`y`**, and **`band`**, each with required CF attributes:
  - `x`: `standard_name = "projection_x_coordinate"`, `units = "metre"`
  - `y`: `standard_name = "projection_y_coordinate"`, `units = "metre"`
  - `band`: `standard_name = "sensor_band_identifier"`, no units required

Each data variable **must** include:
- A valid CF **`standard_name`** (e.g. `surface_reflectance`)
- A corresponding **`units`** attribute
- A **`grid_mapping`** attribute referencing the spatial reference variable

The **grid mapping variable** is required and defines the projection using CF-compliant attributes. The variable name **`spatial_ref`** is recommended for consistency.

---

### 🔁 Optional Spectral Mapping

To support spectral metadata, the dataset **may include** an auxiliary coordinate variable:
- `wavelength(band)` with `standard_name = "radiation_wavelength"`, `units = "micrometre"`

When present, this auxiliary variable **must be referenced** in the data variable using the `coordinates = "wavelength"` attribute. This allows:
- Forward mapping from `band → wavelength`
- Reverse lookup from `wavelength → band`

This pattern aligns with CF conventions for auxiliary coordinates and enables spectral interpretation without requiring `wavelength` as a core dimension.

---

### Additional information:
- The group may contain nested groups, allowing namespacing or grouping of related variables.
- Coordinates can be provided either as explicit arrays (`x`, `y`) or via an affine transform, if supported (subject to profile implementation discussion).
- Chunking layout is implementation-dependent and not prescribed by the profile. The focus is on structure, naming, and semantic metadata for CF compliance.

In [13]:
import xarray as xr
band = xr.open_zarr("./band-raster.zarr", consolidated=False)
band

<xarray.Dataset> Size: 1MB
Dimensions:      (band: 8, y: 128, x: 128)
Coordinates:
  * band         (band) int64 64B 0 1 2 3 4 5 6 7
    wavelength   (band) float64 64B dask.array<chunksize=(8,), meta=np.ndarray>
  * x            (x) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
  * y            (y) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
Data variables:
    reflectance  (band, y, x) float64 1MB dask.array<chunksize=(4, 64, 128), meta=np.ndarray>
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    profile:      band-raster

### 📄 **Profile : `time-series-raster`**

The `time-series-raster` profile defines an extension to existing 2D raster profiles (such as `scalar-raster`, `rgb-raster`,`multispectral-raster` or `band-raster`) to support **time-varying raster datasets**. It applies to any Zarr-encoded geospatial dataset where time is introduced as an additional dimension, enabling representation of dynamic Earth Observation or environmental data.

The dataset **must contain**:
- A global attribute **`Conventions = "CF-1.8"`** (mandatory)
- A global attribute **`profile`** as a list that includes:
  - **`"time-series-raster"`** (mandatory for this profile)
  - At least one base profile such as **`"scalar-raster"`**, **`"rgb-raster"`**, `multispectral-raster, or **`"band-raster"`**

The dataset **must define**:
- A **`time`** coordinate variable with:
  - `standard_name = "time"`
  - ISO 8601 values or datetime64 encoding
  - Units inferred by the encoder (e.g. `"seconds since 1970-01-01T00:00:00Z"`)

- A data variable (e.g. `temperature`, `reflectance`) with dimensions including `time`, in the form:
  - `(time, y, x)` for scalar variables
  - `(time, band, y, x)` or `(time, wavelength, y, x)` for multi-band data

- Spatial coordinate variables `x`, `y` and a **grid mapping variable** (e.g. `spatial_ref`) as required by the referenced base profile.

---

### Additional Information:
- The **`time`** dimension is considered orthogonal to the spatial grid and can support regular or irregular time series.
- This profile is compatible with CF's discrete sampling features and assumes CF encoding conventions for time.
- The group may include nested subgroups for logical separation of time slices or derived outputs.
- Coordinates (`x`, `y`) may be defined explicitly or inferred from an affine transform, depending on implementation support.

Chunking strategy remains implementation-specific. The profile focuses on metadata and dimensional structure required for temporal interoperability in CF-compliant geospatial workflows.

In [15]:
import xarray as xr
xyt = xr.open_zarr("./xyt-raster.zarr", consolidated=False)
xyt

<xarray.Dataset> Size: 526kB
Dimensions:      (time: 4, y: 128, x: 128)
Coordinates:
  * time         (time) datetime64[ns] 32B 2020-01-01 2020-01-02 ... 2020-01-04
  * x            (x) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
  * y            (y) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
Data variables:
    spatial_ref  int64 8B ...
    temperature  (time, y, x) float64 524kB dask.array<chunksize=(2, 128, 128), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8
    profile:      ['time-series-raster', 'scalar-raster']

#### XYWT Example

The time-series-raster profile can be added to any core raster profile such as the multispectral raster profile.

To be discussed and improved: order of dimensions.

In [16]:
import xarray as xr
xywt = xr.open_zarr("./xywt-raster.zarr", consolidated=False)
xywt

<xarray.Dataset> Size: 4MB
Dimensions:      (time: 4, wavelength: 8, y: 128, x: 128)
Coordinates:
  * time         (time) datetime64[ns] 32B 2020-01-01 2020-01-02 ... 2020-01-04
  * wavelength   (wavelength) float64 64B 0.443 0.49 0.56 ... 1.61 2.2 0.865
  * x            (x) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
  * y            (y) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
Data variables:
    reflectance  (time, wavelength, y, x) float64 4MB dask.array<chunksize=(2, 4, 64, 128), meta=np.ndarray>
    spatial_ref  int64 8B ...
Attributes:
    Conventions:  CF-1.8
    profile:      ['time-series-raster', 'multispectral-raster']

### 📄 **Profile: `vertical-raster`**

The `vertical-raster` profile defines a CF-compliant structure for raster datasets in Zarr format, where each layer corresponds to a discrete vertical position such as **altitude** or **depth**. It is designed for representing 3D raster data with a consistent spatial grid across vertical levels, supporting use cases in atmospheric, oceanographic, or elevation-based analysis.

The dataset **must contain**:
- A global attribute **`Conventions = "CF-1.8"`** (mandatory) and **`profile = "vertical-raster"`** (recommended) to identify the profile.
- A data variable (e.g. **`temperature`**) representing the raster array, with dimensions `(z, y, x)`.
- Coordinate variables named **`x`**, **`y`**, and **`z`**, each with required CF attributes:
  - `x`: `standard_name = "projection_x_coordinate"`, `units = "metre"`
  - `y`: `standard_name = "projection_y_coordinate"`, `units = "metre"`
  - `z`: `standard_name = "altitude"` (or a CF-recognised vertical name such as `depth`), `units = "metre"`

Each data variable **must** include:
- A valid CF **`standard_name`** (e.g. `air_temperature`)
- A corresponding **`units`** attribute
- A **`grid_mapping`** attribute referencing the spatial reference variable

The **grid mapping variable** is required and defines the projection using CF-compliant attributes. The variable name **`spatial_ref`** is recommended for consistency.

---

### Additional information:
- The `z` coordinate may represent altitude, height above ground, or depth, depending on the variable context. If applicable, it **should** include the CF attribute `positive = "up"` or `positive = "down"`.
- The group may contain nested groups, allowing namespacing or grouping of related variables.
- Coordinates can be provided either as explicit arrays (`x`, `y`) or via an affine transform, if supported (subject to profile implementation discussion).
- Chunking layout is implementation-dependent and not prescribed by the profile. The focus is on structure, naming, and semantic metadata for CF compliance.

In [17]:
import xarray as xr

xyz = xr.open_zarr("./xyz-raster.zarr", consolidated=False)
xyz

<xarray.Dataset> Size: 657kB
Dimensions:      (z: 5, y: 128, x: 128)
Coordinates:
  * x            (x) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
  * y            (y) float64 1kB 0.0 1.0 2.0 3.0 4.0 ... 124.0 125.0 126.0 127.0
  * z            (z) int64 40B 0 1000 2000 3000 4000
Data variables:
    spatial_ref  int64 8B ...
    temperature  (z, y, x) float64 655kB dask.array<chunksize=(3, 64, 128), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8
    profile:      vertical-raster